In [1]:
import os
import pycocotools.coco as coco

from pycocotools.cocoeval import COCOeval

In [2]:
split = 'val'
annot_path = os.path.join('F:\TUM Learning Material\Forschung\CenterNet\CenterNet-master\data\coco', 'annotations','instances_{}2017.json').format(split)
#initialize coco object given file_path
coco = coco.COCO(annot_path)

detr_dets = coco.loadRes('{}\detr_detection_results.json'.format('F:\TUM Learning Material\Forschung\detr\detr_master\outputs'))

loading annotations into memory...
Done (t=0.75s)
creating index...
index created!
Loading and preparing results...
DONE (t=7.29s)
creating index...
index created!


In [3]:
coco_eval = COCOeval(coco, detr_dets, "bbox")

In [4]:
coco_eval.evaluate()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=61.88s).


In [5]:
result = coco_eval.accumulate()

Accumulating evaluation results...
DONE (t=8.35s).


In [7]:
result['counts']

[10, 101, 80, 4, 3]

In [8]:
result['precision'].shape

(10, 101, 80, 4, 3)

In [6]:
coco_eval.summarize()

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.420
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.624
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.442
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.458
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.611
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.333
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.533
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.574
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.312
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.628
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.805


In [9]:
T, R, K, A, M = result['counts']

In [10]:
precision_matrix = result['precision']

In [11]:
import numpy as np

In [12]:
#maxdetection=100 --> last_index = 2
#small area 
def get_class_precision(precision_matrix, area_idx, maxDet_idx, T=10, R=101, K=80):
    class_precision = precision_matrix[:,:,:,area_idx,maxDet_idx]
    class_precision = class_precision[class_precision>-1].reshape(T,R,K)
    class_precision = np.mean(class_precision, axis=0)
    class_precision = np.mean(class_precision, axis=0)
    return np.round(class_precision, decimals=3) 

In [13]:
smallarea_precision = get_class_precision(precision_matrix, area_idx=1, maxDet_idx=1)
middlearea_precision = get_class_precision(precision_matrix, area_idx=2, maxDet_idx=1)
largearea_precision = get_class_precision(precision_matrix, area_idx=3, maxDet_idx=1)
total_precision = get_class_precision(precision_matrix, area_idx=0, maxDet_idx=1)

In [14]:
class_name = ['person', 'bicycle', 'car', 'motorcycle', 'airplane','bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
      'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
      'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
      'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
      'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
      'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass',
      'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
      'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
      'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
      'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
      'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
      'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [15]:
import sys
from prettytable import PrettyTable

In [16]:
class_precision_table = PrettyTable(['class','small','middle','large','total'])

for cat in range(len(class_name)):
    class_precision_table.add_row([class_name[cat],smallarea_precision[cat],middlearea_precision[cat],largearea_precision[cat],total_precision[cat]])

In [17]:
print(class_precision_table)

+----------------+-------+--------+-------+-------+
|     class      | small | middle | large | total |
+----------------+-------+--------+-------+-------+
|     person     | 0.235 | 0.562  | 0.776 |  0.49 |
|    bicycle     | 0.138 | 0.361  | 0.648 | 0.306 |
|      car       |  0.24 | 0.581  | 0.738 | 0.384 |
|   motorcycle   | 0.167 | 0.386  |  0.66 | 0.436 |
|    airplane    | 0.589 |  0.63  | 0.748 | 0.671 |
|      bus       | 0.217 | 0.494  | 0.828 | 0.655 |
|     train      | 0.417 | 0.232  | 0.711 | 0.659 |
|     truck      | 0.166 | 0.356  | 0.556 | 0.368 |
|      boat      | 0.112 |  0.28  | 0.495 | 0.218 |
| traffic light  | 0.166 | 0.367  | 0.586 | 0.208 |
|  fire hydrant  | 0.361 | 0.731  | 0.796 | 0.662 |
|   stop sign    | 0.241 | 0.628  |  0.95 | 0.621 |
| parking meter  | 0.132 | 0.374  | 0.657 | 0.406 |
|     bench      |  0.14 | 0.221  | 0.485 | 0.262 |
|      bird      | 0.162 | 0.546  | 0.798 |  0.31 |
|      cat       | 0.064 | 0.656  | 0.802 | 0.759 |
|      dog  

In [18]:
precision_sort_idx = np.argsort(-total_precision, kind='mergesort')
i = 0
for i in range(len(precision_sort_idx)):
    print(class_name[precision_sort_idx[i]])

cat
bear
giraffe
zebra
airplane
dog
fire hydrant
train
bus
toilet
laptop
elephant
stop sign
horse
refrigerator
microwave
tv
frisbee
skateboard
mouse
keyboard
pizza
cow
bed
teddy bear
sheep
person
couch
tennis racket
clock
motorcycle
parking meter
oven
bowl
suitcase
snowboard
cup
sandwich
umbrella
car
donut
sports ball
surfboard
fork
truck
sink
cake
kite
hot dog
toaster
scissors
baseball bat
baseball glove
dining table
wine glass
vase
bird
bicycle
tie
cell phone
bottle
potted plant
skis
orange
chair
bench
remote
hair drier
boat
traffic light
broccoli
banana
apple
toothbrush
carrot
spoon
knife
backpack
handbag
book


In [ ]:
#maxdetection=100 --> last_index = 2
#small area 
def get_class_recall(recall_matrix, area_idx, maxDet_idx, T=10, K=80):
    recall_matrix = recall_matrix[:,:,area_idx,maxDet_idx]
    recall_matrix = recall_matrix[recall_matrix>-1].reshape(T,K)
    recall_matrix = np.mean(recall_matrix, axis=0)
    return np.round(recall_matrix, decimals=3) 

In [ ]:
recall_matrix = result['recall']

In [ ]:
recall_matrix.shape

In [ ]:
smallarea_recall = get_class_recall(recall_matrix, area_idx=1, maxDet_idx=1)
middlearea_recall = get_class_recall(recall_matrix, area_idx=2, maxDet_idx=1)
largearea_recall = get_class_recall(recall_matrix, area_idx=3, maxDet_idx=1)
total_recall = get_class_recall(recall_matrix, area_idx=0, maxDet_idx=1)

In [ ]:
class_recall_table = PrettyTable(['class','small','middle','large','total'])

for cat in range(len(class_name)):
    class_recall_table.add_row([class_name[cat],smallarea_recall[cat],middlearea_recall[cat],largearea_recall[cat],total_recall[cat]])

In [ ]:
print(class_recall_table)

In [ ]:
recall_sort_idx = np.argsort(-total_recall, kind='mergesort')
i = 0
for i in range(len(recall_sort_idx)):
    print(class_name[recall_sort_idx[i]])